In [1]:
# coding: utf8

import urllib2
import json
import re

from bs4 import BeautifulSoup
from datetime import datetime

def parse_page(soup):
    """
    Does the parsing of each tender
    Input:
    - soup: HTML response parsed by BeautifulSoup4
    Output:
    - List of strings. Each string is a semi-column-separated list of tender attributes
    """
    # Get the table with tenders
    tenders = soup.find_all("table", class_="tender_table")

    # Get its rows. Each row is another tender except the first row (headers)
    rows = tenders[0].find_all("tr")
    
    result = []
    status_dict = {"accepted" : u"Приём заявок", "protocol" : u"Формирование протоколов",\
                  "opened" : u"Проведена", "canceled" : u"Отклонена нах!"}

    for r in rows:
        output = u"";

        # Tender date
        val = r.find(True, class_="tender_date")
        if (val):
            output = output + val.get_text()
        
        # Tender number
        val = r.find(True, class_="tender_number")
        if (val):
            output = output + ";" + val.get_text()

        # Tender name
        val = r.find(True, class_="tender_name")
        if (val):
            output = output + ";\"" + val.get_text() + "\""
            
        # Tender status
        val = r.find(True, class_="tender_status")
        if (val):
            print val['class'][1], status_dict[val['class'][1]]
            output = output + ";\"" + status_dict[val['class'][1]] + "\""

        # Tender branch
        val = r.find(True, class_="tender_organizator")
        if (val):
            output = output + ";\"" + val.get_text() + "\""

        # Tender subission date
        val = r.find(True, class_="tender_end-date")
        if (val):
            if (len(val.get_text()) > 3):
                output = output + ";" + val.get_text()
            else:
                output = output + ";"
                
        # Tender price
        val = r.find(True, class_="tender_price")
        if (val):
            price = val.get_text()
            price = price.replace(" ", "")
            output = output + ";" + price[:-1] + "\n"
           
        if (len(output) > 0):
            result.append(output)

    return result

# Load text file with OKDP codes
with open('OKDP.txt', 'r') as f:
    content = f.readlines()
    
# Cut the final \n character
okdp_codes = [item[:-1] for item in content]  

In [2]:
req_body = "http://zakupki.rostelecom.ru"

#req_suffix = "/procedure/?arrFilter_ff%5BNAME%5D=&arrFilter_pf%5BRECEIVER_ORG_CODE%5D=&arrFilter_pf%5BOKDP_CODE%5D=&arrFilter_pf%5BOKVED_CODE%5D=&arrFilter_pf%5BCREATION_DATE_FROM%5D=27.09.2016&arrFilter_pf%5BCREATION_DATE_TO%5D=29.09.2016&arrFilter_pf%5BAPPLY_END_DATE_FROM%5D=&arrFilter_pf%5BAPPLY_END_DATE_TO%5D=&set_filter=Y&set_filter=%CF%F0%E8%EC%E5%ED%E8%F2%FC"
req_suffix_1 = "/procedure/?arrFilter_ff%5BNAME%5D=&arrFilter_pf%5BRECEIVER_ORG_CODE%5D=&arrFilter_pf%5BOKDP_CODE%5D="
req_suffix_2 = "&arrFilter_pf%5BOKVED_CODE%5D=&arrFilter_pf%5BCREATION_DATE_FROM%5D=01.01.2016&arrFilter_pf%5BCREATION_DATE_TO%5D=29.09.2016&arrFilter_pf%5BAPPLY_END_DATE_FROM%5D=&arrFilter_pf%5BAPPLY_END_DATE_TO%5D=&set_filter=Y&set_filter=%CF%F0%E8%EC%E5%ED%E8%F2%FC"

with open('tenders.csv', 'w') as f:
    for okdp in okdp_codes:
        # Get the very first page of response  
        response = urllib2.urlopen(req_body + req_suffix_1 + okdp + req_suffix_2)
        result = response.read()
        result = result.decode('windows-1251')

        # Parse the page
        soup = BeautifulSoup(result, "lxml")
        # Get list of tenders on the page
        current_page_tenders = parse_page(soup)

        # Write to file
        for t in current_page_tenders:
            t = okdp + ";" + t
            f.write(t.encode('utf-8'))

        # Proceed to the next pages of response
        refs = soup.find_all("a")
        # Loop through the links to other pages
        for r in refs:
            lnk = r.get('href')

            if ("PAGEN" in lnk):
                # Get next page
                response = urllib2.urlopen(req_body + lnk)
                result = response.read()
                result = result.decode('windows-1251')

                # Parse it
                soup = BeautifulSoup(result, "lxml")
                # Get list of tenders on the page
                current_page_tenders = parse_page(soup)

                # Write to file
                for t in current_page_tenders:
                    t = okdp + ";" + t
                    f.write(t.encode('utf-8'))    


opened Проведена
opened Проведена
opened Проведена
accepted Приём заявок
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
protocol Формирование протоколов
opened Проведена
opened Проведена
protocol Формирование протоколов
opened Проведена
opened Проведена
opened Проведена
opened Проведена
opened Проведена
accepted Приём заявок
protocol Формирование протоколов
accepted Приём заявок
protocol Формирован

error: [Errno 10054] An existing connection was forcibly closed by the remote host